<a href="https://colab.research.google.com/github/JuanArocaMIAR/03MIAR---Algoritmos-de-Optimizacion---2023/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Juan_Aroca_Perez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Juan Aroca Pérez  <br>
Url: https://github.com/JuanArocaMIAR/03MIAR---Algoritmos-de-Optimizacion---2023.git<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema: "Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. "







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

**Representación del espacio de soluciones:**
El espacio de soluciones se puede representar como una matriz binaria, donde las filas representan actores y las columnas representan tomas. Cada columna de la matriz indica si el actor participó en el plano (1) o no (0).

**Función objetivo:**
El objetivo es reducir los costos operativos de los actores de voz. Esto se puede calcular sumando el número de días necesarios para todos los rodajes y luego multiplicando por el precio fijo por jugador por día.

**Implementación de restricciones:**

- Emparejamiento de actores en una escena: para garantizar que los actores en una escena sean compatibles cuando los personajes aparecen juntos, podemos usar una matriz de proximidad para mostrar la relación entre los personajes. Por tanto, podemos comprobar en cada plano que los actores están contentos con estas conexiones.

- Límite de capacidad diaria: Para no exceder las 6 descargas por día, los elementos adquiridos se pueden dividir en 6 grupos, y en base a esto se adquirirá cada grupo y se determinarán los participantes de cada grupo.

In [ ]:
#Respuesta


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
#Respuesta


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [ ]:
#Respuesta